## This notebook shows how you can use the `fastplotlib` API to link `Graphic` objects together

In [1]:
import numpy as np
from fastplotlib.layouts import GridPlot
from fastplotlib.graphics import *
from mesmerize_core import *
from fastplotlib import Plot
from ipywidgets import IntSlider, VBox, Layout
from fastplotlib.graphics.line_slider import LineSlider

2023-01-25 13:08:39.394997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 13:08:39.496128: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-25 13:08:39.498966: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/caitlin/venvs/mescore/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-25

In [2]:
set_parent_raw_data_path("/home/caitlin/caiman_data/")

PosixPath('/home/caitlin/caiman_data')

In [3]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

In [4]:
df = load_batch(batch_path)

In [5]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (...",{'mean-projection-path': a369b894-cd04-46b0-b4...,2023-01-25T11:00:13,2023-01-25T11:00:39,20.57 sec,None,a369b894-cd04-46b0-b4fc-21eb211012a5
1,cnmf,Sue_2x_3000_40_-46,a369b894-cd04-46b0-b4fc-21eb211012a5/a369b894-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 9fd0507e-e4df-4e6b-8a...,2023-01-25T11:02:19,2023-01-25T11:03:02,32.16 sec,None,9fd0507e-e4df-4e6b-8a4b-5c9f9d57df92


In [6]:
contours, coms = df.iloc[-1].cnmf.get_contours("good", swap_dim=False)

input_movie = df.iloc[-1].caiman.get_input_movie()

temporal = df.iloc[-1].cnmf.get_temporal("good")

In [7]:
rand_colors = np.random.rand(len(contours), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 # set alpha = 1

In [8]:
movie_graphic = ImageGraphic(data=input_movie[0], cmap="gnuplot2")
contour_graphic = LineCollection(data=contours, colors=rand_colors)
temporal_graphic = LineCollection(data=temporal, colors=rand_colors)
temporal_stack = LineStack(data=temporal, colors=rand_colors)

In [9]:
grid_plot = GridPlot((1, 2),  
                     names=[["contours", "temporal stack"]])

grid_plot["contours"].add_graphic(movie_graphic)
grid_plot[0, 0].add_graphic(contour_graphic)
grid_plot["temporal stack"].add_graphic(temporal_stack)

plot = Plot()
plot.add_graphic(temporal_graphic)

slider = IntSlider(min=0, max=input_movie.shape[0] - 1, value=0, step=1)

_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=slider)
plot.add_graphic(_ls)

def update_frame(change):
    ix = change["new"]
    movie_graphic.data = input_movie[ix]
    
slider.observe(update_frame, "value")

@plot.renderer.add_event_handler("resize")
def update_slider_width(*args):
    width, h = plot.renderer.logical_size
    slider.layout = Layout(width=f"{width}px")

VBox([grid_plot.show(), plot.show(), slider])

RFBOutputContext()

RFBOutputContext()

In [10]:
plot.auto_scale()
plot.camera.scale.x = 0.85
grid_plot["temporal stack"].auto_scale()

In [11]:
temporal_graphic[:].present = False

In [12]:
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

In [13]:
movie_graphic.link(
    "click",
    target=contour_graphic,
    feature="colors", 
    new_data="w", 
    callback=euclidean
)

contour_graphic.link("colors", target=contour_graphic, feature="thickness", new_data=5)

temporal_stack.link("click", target=temporal_stack, feature="colors", new_data="w")
temporal_stack.link("colors", target=temporal_stack, feature="thickness", new_data=4)

contour_graphic.link("colors", target=temporal_stack, feature="colors", new_data="w", bidirectional=True)

temporal_stack.link("colors", target=temporal_graphic, feature="present", new_data=True)

temporal_graphic[:].present.add_event_handler(plot.auto_scale)

/home/caitlin/repos/fastplotlib/fastplotlib/graphics/features/_base.py:87: UserWarning: Event handler <bound method Interaction.event_handler of LineCollection @ 0x7fe2c566e5b0
Collection of <71> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")
/home/caitlin/repos/fastplotlib/fastplotlib/graphics/features/_base.py:87: UserWarning: Event handler <bound method Interaction.event_handler of LineStack @ 0x7fe2b0b13970
Collection of <71> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")
